In [2]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.nn as nn  
import torch.optim as optim

class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values
        self.targets = self.data.iloc[:, -1].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)  
        return features, target

csv_file = "./MEFAR_MID_sansEEG.csv"
dataset = CustomDataset(csv_file)

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=7, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True)

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_dim = 7
        self.out_dim = 1
        self.fc1 = nn.Linear(self.in_dim, 4)
        self.fc2 = nn.Linear(4, self.out_dim)
        self.relu = nn.LeakyReLU(negative_slope=0.0001)
        self.tanh=nn.Tanh()
        self.sigmoid=nn.Sigmoid()
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        a1 = self.tanh(self.fc1(x))
        output = self.sigmoid(self.fc2(a1))  # Pas besoin de log_softmax pour la régression
        return output


In [77]:
model = MLP()
criterion = nn.MSELoss()   # Utilisation de MSELoss pour la régression
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Valeur pour le gradient scaling
gradient_scale = 0.005  # Vous pouvez ajuster cette valeur selon vos besoins

for epoch in range(1):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # Squeeze pour éliminer les dimensions inutiles
        loss.backward()
        
        # Vérification des valeurs NaN dans les gradients
        if torch.isnan(loss).any() or any(torch.isnan(p.grad).any() for p in model.parameters()):
            print("Gradient contains NaN values. Exiting training.")
            break
        
        # Mise à l'échelle du gradient
        for param in model.parameters():
            param.grad *= gradient_scale
        
        # Clipping de gradient
        nn.utils.clip_grad_norm_(model.parameters(), 1.5)  # Vous pouvez ajuster la valeur de max_norm selon vos besoins
        
        optimizer.step()
        running_loss += loss.item()
        
        if (i + 1) % 2000 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        if (i==50000):
            break
            
    if torch.isnan(loss).any() or any(torch.isnan(p.grad).any() for p in model.parameters()):
        break

print('Finished Training')


[1,  2000] loss: 0.216
[1,  4000] loss: 0.203
[1,  6000] loss: 0.202
[1,  8000] loss: 0.194
[1, 10000] loss: 0.193
[1, 12000] loss: 0.198
[1, 14000] loss: 0.197
[1, 16000] loss: 0.197
[1, 18000] loss: 0.192
[1, 20000] loss: 0.194
[1, 22000] loss: 0.193
[1, 24000] loss: 0.195
[1, 26000] loss: 0.197
[1, 28000] loss: 0.194
[1, 30000] loss: 0.192
[1, 32000] loss: 0.193
[1, 34000] loss: 0.190
[1, 36000] loss: 0.189
[1, 38000] loss: 0.192
[1, 40000] loss: 0.194
[1, 42000] loss: 0.191


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/autograd/__init__.py:251: UserWarning: Error detected in MseLossBackward0. Traceback of forward call that caused the error:
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310

RuntimeError: Function 'MseLossBackward0' returned nan values in its 0th output.

In [56]:
def test_model(model, test_loader, criterion):
    model.eval()  # Mettre le modèle en mode évaluation
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Désactiver le calcul des gradients pendant l'évaluation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  # Convertir les étiquettes en type float
            test_loss += loss.item()
            # Calcul de la précision
            predicted = torch.round(outputs).squeeze()  # Arrondir les prédictions
            #print(outputs ,"     rechercher : " , labels) 
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    # Afficher les résultats
    avg_loss = test_loss / len(test_loader)
    accuracy = correct / total
    print('Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(avg_loss, accuracy * 100))

# Utiliser la fonction test_model pour évaluer le modèle
test_model(model, test_loader, criterion)


Test Loss: 0.1375, Accuracy: 80.53%


In [15]:
# Supposons que 'model' soit votre modèle PyTorch
# Vous pouvez afficher les poids des neurones comme ceci :
for name, param in model.named_parameters():
    if 'weight' in name:
        print(name, param.data)

fc1.weight tensor([[-0.9875,  4.5964,  3.7591, -1.4531,  1.3666,  0.1604, -0.9809],
        [ 0.5353, -1.2062,  1.5831, -0.3493,  0.7210, -0.6903,  6.5938],
        [ 0.0360, -1.6728,  3.2348, -0.4110, -0.9494,  0.3122,  4.3936],
        [-0.3583, -3.8716,  4.8726, -0.2780,  0.3309, -0.0482,  1.7401]])
fc2.weight tensor([[ 5.7486, -3.6492,  3.5809, -5.5602]])
